In [ ]:
''' ####label 기억용 개인간 차이 있음
# 0 : 스트레칭 1번 팔 당기기
  1 : 스트레칭 2번 옆구리
  2 : O 표시
  3 : 가만히 있기
  4 : X 표시
  5 : 팔벌려뛰기
  6 : 한 발 서기
  7 : 스쿼트
  8 : 런지
'''

In [9]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [10]:
actions = [
    '0',
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8'
]

data = np.concatenate([
    np.load('./dataset/seq_0.npy'),
    np.load('./dataset/seq_1.npy'),
    np.load('./dataset/seq_2.npy'),
    np.load('./dataset/seq_3.npy'),
    np.load('./dataset/seq_4.npy'),
    np.load('./dataset/seq_5.npy'),
    np.load('./dataset/seq_6.npy'),
    np.load('./dataset/seq_7.npy'),
    np.load('./dataset/seq_8.npy'),
], axis=0)

data.shape

(14325, 30, 170)

In [11]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(14325, 30, 169)
(14325,)


In [12]:
print(np.unique(labels))


[0. 1. 2. 3. 4. 5. 6. 7. 8.]


In [13]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(14325, 9)

In [14]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(12892, 30, 169) (12892, 9)
(1433, 30, 169) (1433, 9)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        59,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 9)              │           297 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,281 (243.29 KB)

 Trainable params: 62,281 (243.29 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.keras', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)


Epoch 1/200
398/403 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.1596 - loss: 100.7665
Epoch 1: val_acc improved from -inf to 0.32449, saving model to models/model.keras
403/403 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - acc: 0.1603 - loss: 100.1200 - val_acc: 0.3245 - val_loss: 9.2032 - learning_rate: 0.0010
Epoch 2/200
398/403 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.3236 - loss: 7.2055
Epoch 2: val_acc improved from 0.32449 to 0.39986, saving model to models/model.keras
403/403 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - acc: 0.3236 - loss: 7.1872 - val_acc: 0.3999 - val_loss: 3.4612 - learning_rate: 0.0010
Epoch 3/200
402/403 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.4150 - loss: 3.2917
Epoch 3: val_acc did not improve from 0.39986
403/403 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - acc: 0.4151 - loss: 3.2903 - val_acc: 0.3943 - val_loss: 3.1044 - learning_rate: 0.0010
Epoch 4/200
400/403 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.4576 - loss: 2.6956
Epoch 4: val_acc improved from 0.39986 to 0.51221, saving mo

KeyboardInterrupt: 